In [1]:
### 고민해봐야할 사항
# 로그인하여서 최근 기록을 바탕으로 가져오는게 빠를것인가
# 코드를 돌려가면서 검색하는게 나을것인가 -> 다른 크롤러와 함께 실행시킨다면 이쪽이 맞는 방안
# 카카오스탁은 페이지 로딩속도가 느린편에 속한다. -> 타임슬립을 자주 넣어주어야 한다.
# 댓글 내용을 가져오기 위해서는 첫번째 글에 댓글표시 버튼을 눌러 해당 글로 화면이 전환되어야 한다.
# 네이버 크롤링과 비슷한 원리로 작동시킨다.
# rngdrngd@dreamwiz.com

In [2]:
import selenium
from selenium import webdriver as wd
from urllib.request import urlopen

import time
from selenium.webdriver.common.by import By
# WebDriverWait는 Selenium 2.4.0 이후 부터 사용 가능합니다.
from selenium.webdriver.support.ui import WebDriverWait
# expected_conditions는 Selenium 2.26.0 이후 부터 사용 가능합니다.
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm_notebook
from datetime import datetime as dt
import datetime
import pandas as pd

In [3]:
def save_csv(data, stock_code):      
        frame = pd.DataFrame(data)
        with open('./kakao_{0}.csv'.format(stock_code), 'a', encoding='utf-8') as f:
            frame.to_csv(f, header=False, index=False, encoding='utf-8')            
#         jungbo.clear()

In [4]:
driver = wd.Chrome('./data/chromedriver.exe')
driver.get('http://stock.kakao.com/m')
data = []
login=driver.find_element_by_css_selector('body > div:nth-child(1) > div > div:nth-child(1) > footer > div > div.btnB > a:nth-child(1)')
if login.text=='로그인':
    login.click()
time.sleep(1)
# alert = driver.switch_to_alert()   
driver.switch_to_window(driver.window_handles[1])  #attitude addictive  
idea = driver.find_element_by_css_selector('#loginEmail')
# 아이디와 비밀번호를 입력하세요
idea.send_keys('')
password = driver.find_element_by_css_selector('#loginPw')
password.send_keys('')
driver.find_element_by_css_selector('#login-form > fieldset > button').click()
driver.switch_to_window(driver.window_handles[0])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: use driver.switch_to.window instead
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: use driver.switch_to.window instead
  from ipykernel import kernelapp as app


In [5]:
def collecting(input_date):
    stime=time.time()
    idx = 0
    while True:
    # 여기부터 본문 내용 크롤링
        data=[]
        pra =[]
        mord=[]
        time.sleep(0.25)
        date = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body > div:nth-child(1) > div > div:nth-child(1) > main > main > article > div.viewB > div.f_clear > div.nickname > em')))
        # 오늘 날짜
        if date.text.find('시간')>=0 or date.text.find('분')>=0 or date.text.find('초')>=0:
            pass
        else:
            # 해당 요일을 넘어가면 종료
            if (dt.strptime('20'+date.text,"%Y.%m.%d").date() - dt.strptime(input_date,"%Y.%m.%d").date()).days < 0 :
                break           
        #본문내용
        text = driver.find_element_by_css_selector('body > div:nth-child(1) > div > div:nth-child(1) > main > main > article > div.viewB > p')
        # 작성자 아이디
        writer = driver.find_element_by_css_selector('body > div:nth-child(1) > div > div:nth-child(1) > main > main > article > div.viewB > div.f_clear > div.nickname')
        writer1 =writer.find_element_by_tag_name('p')
        # 좋아요 싫어요 갯수
        like = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR,'body > div:nth-child(1) > div > div:nth-child(1) > main > main > article > div.viewB > ul.recomm')))
        # 예측분석
        try:
            pred = driver.find_element_by_css_selector('body > div:nth-child(1) > div > div:nth-child(1) > main > main > article > div.viewB > div.f_clear > div.fr > span')
            #             data.append(pred.txt)
        except:
            pass
        else:
            pra.append(pred.text)
        mord = (date.text,writer1.text,text.text,like.text[2],like.text[8],pra)
        data.append(mord)
        save_csv(data, stock_code)
        driver.find_element_by_css_selector('body > div:nth-child(1) > div > div:nth-child(1) > main > main > article > ul > li.ty02 > a > span').click()       
    
    dtime=time.time()


In [6]:
def start_crawling(stock_code):    
    elem = driver.find_element_by_class_name("maintxt")
    # 검색창에 입력한 값이 있을수도 있음으로 클리어처리 한다.
    elem.clear()
    # 종목코드를 입력한다. 이부분은 차후 함수에 변수명이 들어갈 자리이다.
    # 객체화를 시키기 위한 부분은 위에서부터 처리하도록 한다.
    # 여기에서는 sk 하이닉스(000660)로 사용한다.
    elem.send_keys(stock_code)
    # 검색을 실행한다
    # elem.submit ()
    # 동적 페이지이라 submit이 사용되지 않았음..
    # 클릭방식으로 검색을 실행하기로 한다.
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body > div:nth-child(1) > div > div:nth-child(1) > header > div.mainSearch > em > div > button'))).click()
    #driver.find_element_by_css_selector('body > div:nth-child(1) > div > div:nth-child(1) > header > div.mainSearch > em > div > button').click()
    # 두번째 클릭이 필요한 화면이 발생 다시한번더 클릭
    #time.sleep(1)
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body > div:nth-child(1) > div > div:nth-child(1) > main > article > div > div.contW01.pb0 > div.myStkB > div > ul > li > a.addFavW'))).click()
    #driver.find_element_by_css_selector('body > div:nth-child(1) > div > div:nth-child(1) > main > article > div > div.contW01.pb0 > div.myStkB > div > ul > li > a.addFavW').click()
    # 토론방으로 이동
    time.sleep(1)
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'body > div:nth-child(1) > div > div:nth-child(1) > header > div:nth-child(4) > div > div > nav > ul > li:nth-child(2) > a'))).click()
    #driver.find_element_by_css_selector('body > div:nth-child(1) > div > div:nth-child(1) > header > div:nth-child(4) > div > div > nav > ul > li:nth-child(2) > a').click()
    # 첫번째 글로 이동하기
    time.sleep(1)
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR,'body > div:nth-child(1) > div > div:nth-child(1) > main > article > div.discussW > div > ul > span > li:nth-child(1) > ul > li.ty03'))).click()
    #driver.find_element_by_css_selector('body > div:nth-child(1) > div > div:nth-child(1) > main > article > div.discussW > div > ul > span > li:nth-child(1) > ul > li.ty03').click()

In [9]:
### 초기화
#크롤링 임시 그릇 -> 모니터링을 위해 글로벌 변수로 뺏음.
data = [] 
# 마감일 
deadline_date = '2018.11.02'
# 수집된 글의 일자
collected_date = set() # 지역 변수는 외부에서 꺼내보기가 쉽지 않다.
# 리스트 
stock_code_list=['000660']
stock_code=''
# driver = wd.Chrome('./data/chromedriver.exe')
# login()

for present_code in stock_code_list:
    stock_code=present_code
    start_crawling(stock_code)
    collecting(deadline_date)
    driver.find_element_by_css_selector('body > div:nth-child(1) > div > div:nth-child(1) > header > div:nth-child(1) > div > div > span > a.btnHome').click()


WebDriverException: Message: chrome not reachable
  (Session info: chrome=72.0.3626.119)
  (Driver info: chromedriver=2.42.591088 (7b2b2dca23cca0862f674758c9a3933e685c27d5),platform=Windows NT 10.0.17763 x86_64)
